In [ ]:
# Install all the required libraries
!pip install numpy pandas scikit-learn statsmodels econml lightgbm matplotlib seaborn dowhy

In [ ]:
# Necessary imports

import numpy as np
import pandas as pd
import statsmodels.api as sm
from lightgbm import LGBMRegressor, LGBMClassifier
from econml.dml import LinearDML, CausalForestDML
from econml.metalearners import TLearner, SLearner, XLearner
from econml.inference import BootstrapInference
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import dowhy
import dowhy.datasets

warnings.filterwarnings('ignore')

# Bootstrap replicates for inference (small for speed, increase for better CIs)
breps = 10

## Causal ML:

Causal ML particularly useful when:

1. Large number of features
2. Complex functional form of Confounders.

Let's set up the following simulated data to showcase the advantages of Causal ML, more specifically Double-Debiased Machine Learning.

## Data Generating Process (DGP)

The simulation follows these key steps:

### 1. Covariates (Features)
First, we generate a matrix of 15 covariates, $\mathbf{X}$, for each of the $N$ individuals. Each covariate is drawn independently from a standard normal distribution:

$$X_{ij} \sim \mathcal{N}(0, 1) \quad \text{for } j = 0, 1, \dots, 99$$

***

### 2. Confounding and Treatment Assignment
A complex, non-linear **confounding term**, $C_i$, is created using the first four covariates. This term will influence both the treatment assignment and the outcome, which is the definition of a confounder.

The confounding term $C_i$ is defined as:
$$C_i = \sin(2X_{i0}) + \cos(2X_{i1}) + X_{i2}^3 + 0.5 X_{i3}^2$$
The **treatment assignment**, $D_i$, is then determined by a Bernoulli trial where the probability of receiving the treatment (the propensity score, $f(\mathbf{X}_i)$) is a logistic function of this confounding term.

$$P(D_i=1 | \mathbf{X}_i) = f(\mathbf{X}_i) = \frac{1}{1 + \exp(-1.5(C_i - 0.5))}$$
So, $D_i \sim \text{Bernoulli}(e(\mathbf{X}_i))$.

***

### 3. Outcome Model
Finally, the **outcome model** for $Y_i$ is a linear combination of the treatment effect and a function of the covariates, $g(\mathbf{X}_i)$, plus a random error term.

The complete outcome model is:$$Y_i = \alpha + \tau D_i + g(\mathbf{X}_i) + \epsilon_i$$
Where the true ATE is $\tau = 10$ and the error term is drawn from a standard normal distribution, $\epsilon_i \sim \mathcal{N}(0, 1)$. 

In [ ]:
# --- 1. Set up a More Robust Simulation ---

true_ate = 10.0
n_samples = 100000
n_features = 15
np.random.seed(123)

X_data = np.random.normal(0, 1, size=(n_samples, n_features))
X = pd.DataFrame(
    X_data,
    columns=[f'X{i}' for i in range(n_features)]
)

# --- 2. Define a DGP with STRONGER Confounding ---

# This non-linear term will now have a much larger effect
confounders = np.sin(X['X0'] * 2) + np.cos(X['X1'] * 2) + (X['X2']**3) + 0.5 * X['X3']**2

# Treatment assignment with confounding
propensity_score = 1 / (1 + np.exp(-1.5 * (confounders - 0.5)))
D = np.random.binomial(1, p=propensity_score)

# Outcome model with strong confounding
# Both treatment assignment AND outcome depend on the same confounders
g_X = 3 * confounders + 0.5 * X['X4']  # Outcome function depends on confounders
Y = true_ate * D + g_X + np.random.normal(0, 1, n_samples)

print("--- Data Simulation Setup ---")
print(f"True Average Treatment Effect (ATE): {true_ate}")
print(f"Sample size: {n_samples:,}")
print(f"Treatment prevalence: {D.mean():.3f}\n")

## OLS Estimation:

Let's first estimate the model with OLS. . The standard OLS approach models the outcome $Y$ as a linear function of the treatment $D$ and the covariates $\mathbf{X}$: $$ Y_i = \alpha + \tau D_i + \boldsymbol{\beta}'\mathbf{X}_i + u_i $$

The coefficient $\tau$ represents the ATE estimate. The highly non-linear DGP mechanically creates a **OVB**, yielding a biased ATE estimate. The effect of **OVB** is ambiguous, i.e. upward or downward bias. 

In [ ]:
# --- 3. Model 1: Naive OLS ---

# RHS of estimation equation
X_with_const = sm.add_constant(np.c_[D, X])

# Instantiate OLS model class and fit the model
ols_model = sm.OLS(Y, X_with_const).fit()

# Extract ATE parameter
ols_ate_estimate = ols_model.params[1]

# The treatment effect is the coefficient for D, which should be index 1 (after constant)
treatment_param_idx = 1
ols_conf_interval = ols_model.conf_int().iloc[treatment_param_idx]

print("--- 1. Naive OLS Results ---")
print(f"Estimated ATE with OLS: {ols_ate_estimate:.4f}")
print(f"OLS 95% Confidence Interval: [{ols_conf_interval[0]:.4f}, {ols_conf_interval[1]:.4f}]")
print(f"Bias: {ols_ate_estimate - true_ate:.4f}")

### OLS Results:

Upward bias in ATE estimates ($\hat{\tau} = 13.808>10$), where the bias is $\hat{\tau} - \tau_{\text{true}} = 3.8$. Now let's look at the result from DML.

***

# Meta-Learners, Causal Forest and Double/Debiased ML

1. **S-Learner (Single Learner)**: Uses a single model with treatment as a feature
2. **T-Learner (Two Learner)**: Trains separate models for treated and control groups  
3. **X-Learner**: Advanced method that combines T-Learner with cross-fitting
4. **Causal Forest**: Tree-based method for heterogeneous treatment effects

Each method has different assumptions and performance characteristics depending on the data structure.

## S-Learner (Single Learner)

The S-Learner approach uses a single machine learning model to predict the outcome $Y$ using both the treatment indicator $D$ and covariates $\mathbf{X}$ as features:

$$\hat{\mu}(d, \mathbf{x}) = \mathbb{E}[Y | D = d, \mathbf{X} = \mathbf{x}]$$

The ATE is then estimated by predicting counterfactual outcomes:
$$\hat{\tau}_{\text{S}} = \frac{1}{N} \sum_{i=1}^{N} \left[ \hat{\mu}(1, \mathbf{X}_i) - \hat{\mu}(0, \mathbf{X}_i) \right]$$

In [ ]:
print("--- EconML S-Learner with Bootstrap Inference ---")

# Create S-learner with bootstrap inference for confidence intervals
# Bootstrap inference provides theoretically sound confidence intervals
s_learner_econml = SLearner(
    overall_model=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    )
)

# Fit with bootstrap inference enabled
# n_bootstrap_samples controls the number of bootstrap samples (higher = more accurate but slower)
s_learner_econml.fit(
    Y=Y, 
    T=D, 
    X=X, 
    inference=BootstrapInference(n_bootstrap_samples=breps)
)

# Calculate ATE point estimate
slearner_ate = s_learner_econml.ate(X=X)

# Get confidence interval for ATE using built-in method
# This uses proper bootstrap inference to compute confidence intervals
slearner_ci_lower, slearner_ci_upper = s_learner_econml.ate_interval(X=X, alpha=0.05)

# Extract individual treatment effects (CATEs) for each observation
slearner_cate = s_learner_econml.effect(X)

print(f"S-learner ATE: {slearner_ate:.4f}")
print(f"95% Confidence Interval: [{slearner_ci_lower:.4f}, {slearner_ci_upper:.4f}]")
print(f"Bias: {slearner_ate - true_ate:.4f}")
print(f"CATE std: {np.std(slearner_cate):.4f}")
print(f"CATE range: [{np.min(slearner_cate):.4f}, {np.max(slearner_cate):.4f}]")

## T-Learner (Two Learner)

The T-Learner approach trains two separate models: one for the treated group and one for the control group:

- **Control model**: $\hat{\mu}_0(x) = \mathbb{E}[Y | D = 0, \mathbf{X} = x]$
- **Treatment model**: $\hat{\mu}_1(x) = \mathbb{E}[Y | D = 1, \mathbf{X} = x]$

The ATE is estimated as:
$$\hat{\tau}(x) = \frac{1}{N} \sum_{i=1}^{N} \left[ \hat{\mu}_1(\mathbf{X}_i) - \hat{\mu}_0(\mathbf{X}_i) \right]$$

In [ ]:
# --- EconML T-Learner with Bootstrap Inference ---

print("--- EconML T-Learner with Bootstrap Inference ---")

# Create T-learner with bootstrap inference for confidence intervals
# Bootstrap inference provides theoretically sound confidence intervals
t_learner_econml = TLearner(
    models=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    )
)

# Fit with bootstrap inference enabled
# n_bootstrap_samples controls the number of bootstrap samples (higher = more accurate but slower)
t_learner_econml.fit(
    Y=Y, 
    T=D, 
    X=X, 
    inference=BootstrapInference(n_bootstrap_samples=breps)
)

# Calculate ATE point estimate
t_learner_ate = t_learner_econml.ate(X=X)

# Get confidence interval for ATE using built-in method
# This uses proper bootstrap inference to compute confidence intervals
t_learner_ci_lower, t_learner_ci_upper = t_learner_econml.ate_interval(X=X, alpha=0.05)

# Extract individual treatment effects (CATEs) for each observation
t_learner_cate = t_learner_econml.effect(X)

print(f"T-learner ATE: {t_learner_ate:.4f}")
print(f"95% Confidence Interval: [{t_learner_ci_lower:.4f}, {t_learner_ci_upper:.4f}]")
print(f"Bias: {t_learner_ate - true_ate:.4f}")
print(f"CATE std: {np.std(t_learner_cate):.4f}")
print(f"CATE range: [{np.min(t_learner_cate):.4f}, {np.max(t_learner_cate):.4f}]")

## X-Learner

The X-Learner is a more sophisticated meta-learner that combines the T-Learner approach with additional modeling steps to reduce bias. It's particularly effective when treatment and control groups have different sizes.

#### The X-Learner procedure:
1. Estimate $\hat{\mu}_0$ and $\hat{\mu}_1$ like in T-Learner
2. Compute **pseudo-outcomes** for treatment: $\tilde{D}_i^1 = Y_i - \hat{\mu}_0(X_i)$ for treated units 
    - Subtract imputed counterfactuals from actual outcomes 
3. Compute **pseudo-outcomes** for control: $\tilde{D}_i^0 = \hat{\mu}_1(X_i) - Y_i$ for control units 
    - Subtract imputed counterfactual from the treated counterfactual from actual outcomes
4. Train 2 models for the treatment areas:
    - **CATE MODEL 1**: $\hat{\tau}_1(x) \leftarrow \text{Fit model on} \left\{(X_i, \tilde{D}^1_i)|D_i = 1 \right\}$
    - **CATE MODEL 2**: $\hat{\tau}_0(x) \leftarrow \text{Fit model on} \left\{(X_i, \tilde{D}^0_i)|D_i = 0 \right\}$
5. Combine estimates using **estimated** propensity scores:
    - **Final CATE Estimate**: $$
\hat{\tau}(x) = \overbrace{\hat{f}(x)}^{\substack{\text{Probability of} \\ \text{being treated}}} \cdot \overbrace{\hat{\tau}_0(x)}^{\substack{\text{CATE model derived from} \\ \text{the NOT treated group}}} + \overbrace{(1-\hat{f}(x))}^{\substack{\text{Probability of} \\ \text{NOT being treated}}} \cdot \overbrace{\hat{\tau}_1(x)}^{\substack{\text{CATE model derived from} \\ \text{the TREATED group}}}$$

**Main Goal**: Weighted average of CATEs where more trust is on the more populated group (great for treatment areas imbalance) $\rightarrow$ overweight more populated group for more confident treatment estimate

In [ ]:
# --- EconML X-Learner with Bootstrap Inference ---

print("--- EconML X-Learner with Bootstrap Inference ---")

# Create X-learner with bootstrap inference for confidence intervals
# Bootstrap inference provides theoretically sound confidence intervals
x_learner_econml = XLearner(
    # Model for Pseudo-outcomes
    models=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    ),
    # Model for Propensity Scores
    propensity_model=LGBMClassifier(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    ),
    # Model CATEs (regress pseudo-outcomes on X in each treatment group)
    cate_models=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    )
)

# Fit with bootstrap inference enabled
# n_bootstrap_samples controls the number of bootstrap samples (higher = more accurate but slower)
x_learner_econml.fit(
    Y=Y, 
    T=D, 
    X=X, 
    inference=BootstrapInference(n_bootstrap_samples=10)
)

# Calculate ATE point estimate
x_learner_ate = x_learner_econml.ate(X=X)

# Get confidence interval for ATE using built-in method
# This uses proper bootstrap inference to compute confidence intervals
x_learner_ci_lower, x_learner_ci_upper = x_learner_econml.ate_interval(X=X, alpha=0.05)

# Extract individual treatment effects (CATEs) for each observation
x_learner_cate = x_learner_econml.effect(X)

print(f"X-learner ATE: {x_learner_ate:.4f}")
print(f"95% Confidence Interval: [{x_learner_ci_lower:.4f}, {x_learner_ci_upper:.4f}]")
print(f"Bias: {x_learner_ate - true_ate:.4f}")
print(f"CATE std: {np.std(x_learner_cate):.4f}")
print(f"CATE range: [{np.min(x_learner_cate):.4f}, {np.max(x_learner_cate):.4f}]")

### Causal Forest

Causal Forest is a tree-based method that can capture heterogeneous treatment effects and complex non-linear relationships. It builds on random forests but is specifically designed for causal inference.

Key features:
- Handles heterogeneous treatment effects naturally
- Non-parametric and flexible
- Provides honest estimates through sample splitting
- Can identify subgroups with different treatment effects

***

### How does it work?

1. Model to predict outcomes $\rightarrow$ regress Y on X and store residuals $\tilde{Y}$
2. Model to predict treatment $\rightarrow$ regress D on X and store residuals $\tilde{D}$
3. Model for treatment effect heterogeneity $\rightarrow$ Causal Trees on the following model $$\tilde{Y} = \tau(X) \tilde{D}$$

### Causal Trees (Athey & Imbens, 2016)

Main goal is to predict a treatment effect $\tau$. 

#### Splitting Logic:
Splits the data to make the treatment effect as different as possible between new groups. 

#### Output: 
Predicted treatment effect (**CATE**): each final ''leaf'' contains a final treatment effect. The **CATE** is defined as: $$\tau(x) = \mathbb{E}\left[Y^{(1)} - Y^{(0)} |X_i = x_i\right]$$

## The algorithm:

1. **Split the data**: Randomly divide the full dataset into a Splitting Sample and an Estimation Sample.
2. Use splitting sample to find best split:
    - Look for every possible split on every feature, e.g. $\text{age}\leq 40$ and $\text{age}>40$
    - Inside each temporary group calculates a rough estimate of the treatment effect: $$\tau = \mathbb{E}\left[Y^{(1)} - Y^{(0)}\right]$$
    - Finally chooses the split maximizing the difference in treatment effects between child nodes

3. Repeat this process recursively, splitting each node until a stopping criterion is met
4. Having finalized the tree structure estimate the **CATE** using **Estimation Sample**



***

### Note on Inference:

Subforest size is used to create parallel processing to estimate the treatment effects. The number of independent forests will then be n_estimators \ subforest_size = 100, effetictevly replicating the same analysis 100 times to estimate variance, similar to a bootstrap procedure.

In [ ]:
# --- EconML Causal Forest ---

print("--- EconML Causal Forest---")

# Causal Forest
causal_forest_econml = CausalForestDML(
    model_y=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    ),
    model_t=LGBMClassifier(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    ),
    
    n_estimators=200, # n_estimators must be divisible by subforest_size
    subforest_size=5,  # Subforests to compute variance 
    max_depth=6,      # Reduced depth for faster execution
    min_samples_split=10,
    min_samples_leaf=5,
    cv=3,  # Cross-fitting folds (reduced for faster execution)
    discrete_treatment=True,
    n_jobs=-1,
    random_state=123
)

# Fit model
causal_forest_econml.fit(
    Y=Y, 
    T=D, 
    X=X, 
)

# Calculate ATE point estimate
forest_ate = causal_forest_econml.ate(X=X)

# Get confidence interval for ATE using built-in method
# This uses proper bootstrap inference to compute confidence intervals
forest_ci_lower, forest_ci_upper = causal_forest_econml.ate_interval(X=X, alpha=0.05)

# Estimate individual treatment effects for heterogeneity analysis
cate_forest = causal_forest_econml.effect(X)

print(f"Causal Forest ATE: {forest_ate:.4f}")
print(f"95% Confidence Interval: [{forest_ci_lower:.4f}, {forest_ci_upper:.4f}]")
print(f"Bias: {forest_ate - true_ate:.4f}")

## Double/Debiased Machine Learning: Estimation

1. Estimate first one flexible model for the outcome only based on confounders ($X_i$) and stores the residuals: $$ \tilde{Y} = Y - Y_{\text{predicted}}\text{.}$$

2. Trains a separate model to predict the treatment $D$ on the same confounders and calculates the residuals: $$\tilde{D} = D - D_{\text{prediction}}\text{.}$$ These residuals represent the part of the treatment that is not explained by the confounders.

By doing so we remove the effect of confounders on both the **outcome** and **treatment assignment**

**CAVEAT**: To avoid overfitting use **K-Fold crossfitting** for the predictions, split data in K-fold and use K-1 fold for training and last fold for predictions iteratively.
***

## Final Step:

**Debiased Estimation**: Run a simple linear regression on just the residuals $\tilde{Y}$ and $\tilde{D}$: $$\tilde{Y_i} = \beta_0 + \tau\tilde{D_i} + u_i.$$ Now $\hat{\tau}$ is our ATE coefficient.

In [ ]:
# --- 4. Model 2: Double Machine Learning ---

# Instantiating the K-Fold cross-validator
cv_splitter = KFold(n_splits=5, shuffle=True, random_state=123)

# Instantiate the LinearDML model with LightGBM
est_dml = LinearDML(
    model_y=LGBMRegressor(
        n_estimators=100, 
        num_leaves=30, 
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    ),
    model_t=LGBMClassifier(
        n_estimators=100, 
        num_leaves=30, 
        learning_rate=0.1,
        random_state=123,
        verbosity=-1
    ),
    # Cross-fitting
    cv=cv_splitter,
    discrete_treatment=True,
    # Replication seed
    random_state=123
)

est_dml.fit(
    Y, T=D, X=X
)

ate_summary = est_dml.ate_inference(X=X)
dml_ate_estimate = ate_summary.mean_point
dml_ci_lower, dml_ci_upper = ate_summary.conf_int_mean(alpha=0.05)

# Extract individual treatment effects (CATEs) for each observation
dml_cate = est_dml.effect(X)

print("--- 2. Double Machine Learning Results ---")
print(f"Estimated ATE: {dml_ate_estimate:.4f}")
print(f"P-value: {ate_summary.pvalue():.6f}")
print(f"95% Confidence Interval: [{dml_ci_lower:.4f}, {dml_ci_upper:.4f}]")
print(f"Bias: {dml_ate_estimate - true_ate:.4f}")
print(f"CATE std: {np.std(dml_cate):.4f}")
print(f"CATE range: [{np.min(dml_cate):.4f}, {np.max(dml_cate):.4f}]")

## Visualize the Results

In [ ]:
# --- 5. Visualize Results ---

plt.style.use('default')
sns.set_style("whitegrid")

fig, ax = plt.subplots(figsize=(14, 8))

# Data for plotting - all methods with updated variable names
methods = ['OLS', 'S-Learner', 'T-Learner', 'X-Learner', 'Causal Forest', 'DML']
estimates = [
    ols_ate_estimate, 
    slearner_ate, 
    t_learner_ate, 
    x_learner_ate, 
    forest_ate, 
    dml_ate_estimate
]
ci_lowers = [
    ols_conf_interval[0], 
    slearner_ci_lower, 
    t_learner_ci_lower, 
    x_learner_ci_lower, 
    forest_ci_lower, 
    dml_ci_lower
]
ci_uppers = [
    ols_conf_interval[1], 
    slearner_ci_upper, 
    t_learner_ci_upper, 
    x_learner_ci_upper, 
    forest_ci_upper, 
    dml_ci_upper
]

# Color scheme: Using seaborn Pastel2 palette for better aesthetics
colors = sns.color_palette("Set2", len(methods))

# Create horizontal error bars
y_positions = list(range(len(methods)))
for i, (method, est, ci_low, ci_up, color) in enumerate(zip(methods, estimates, ci_lowers, ci_uppers, colors)):
    # Error bar
    ax.errorbar(est, y_positions[i], 
                xerr=[[est - ci_low], [ci_up - est]], 
                fmt='o', color=color, capsize=8, capthick=2,
                markersize=10, linewidth=2, label=f'{method}: {est:.3f}')

# Add vertical line for true ATE
ax.axvline(x=true_ate, color='black', linestyle='--', linewidth=1, 
           label=f'True ATE: {true_ate}', alpha=0.8)

# Customize the plot
ax.set_yticks(y_positions)
ax.set_yticklabels(methods, fontsize=12)
ax.set_xlabel('Treatment Effect Estimate', fontsize=14, fontweight='bold')
ax.set_title('Comparison of Causal Inference Methods: Treatment Effect Estimates', 
             fontsize=16, fontweight='bold', pad=20)

# Remove top and right spines
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Add grid for better readability
ax.grid(True, alpha=0.3, axis='x')

# Add legend
ax.legend(loc='upper right', fontsize=10, frameon=True, fancybox=True, shadow=True)

# Set x-axis limits to better show the confidence intervals
all_ci_values = ci_lowers + ci_uppers
x_min = min(all_ci_values) - 1
x_max = max(all_ci_values) + 1
ax.set_xlim(x_min, x_max)

# Add some padding to y-axis
ax.set_ylim(-0.5, len(methods) - 0.5)

plt.tight_layout()
plt.show()

# Print summary table
print("\n" + "="*80)
print("SUMMARY: Causal Inference Methods Comparison")
print("="*80)
print(f"{'Method':<15} {'ATE Estimate':<12} {'Bias':<8} {'95% CI Lower':<12} {'95% CI Upper':<12}")
print("-"*80)
for method, est, ci_low, ci_up in zip(methods, estimates, ci_lowers, ci_uppers):
    bias = est - true_ate
    print(f"{method:<15} {est:<12.4f} {bias:<8.4f} {ci_low:<12.4f} {ci_up:<12.4f}")
print(f"{'True ATE':<15} {true_ate:<12.4f} {'0.0000':<8} {'':<12} {'':<12}")
print("="*80)

***
# CATEs: 
## Focus on Conditional Average Treatment Effects

# Real-World Application: The Lalonde NSW Dataset

Now let's apply our causal ML methods to **real data** to see how they perform in practice. We'll use the famous **National Supported Work (NSW) dataset** analyzed by Robert Lalonde.

## The Dataset: Job Training Program Evaluation

The NSW program was a job training initiative from the 1970s designed to help disadvantaged workers. The key research question is:

**Did participation in the job training program causally increase participants' earnings?**

### Key Variables:
- **Treatment (`treat`)**: Binary indicator (1 = participated in program, 0 = control group)
- **Outcome (`re78`)**: Real earnings in 1978 (post-program)
- **Covariates**: Pre-treatment characteristics including:
  - Demographics: `age`, `educ` (education), `black`, `hisp`, `married`
  - Employment history: `re74`, `re75` (earnings in 1974-1975)
  - Education status: `nodegree` (no high school degree)

This is a **classic causal inference dataset** where we expect **genuine heterogeneous treatment effects** - the program likely helps some individuals more than others based on their characteristics.

In [ ]:
# Load the Lalonde dataset
df_lalonde = dowhy.datasets.lalonde_dataset()

print("=== LALONDE NSW DATASET OVERVIEW ===")
print(f"Dataset shape: {df_lalonde.shape}")
print(f"\nDataset information:")
df_lalonde.info()

# Define variables for the Lalonde dataset
outcome_lalonde = 're78'      # Real earnings in 1978 (outcome)
treatment_lalonde = 'treat'   # 1 if in the program, 0 otherwise (treatment)
covariates_lalonde = [col for col in df_lalonde.columns if col not in [outcome_lalonde, treatment_lalonde]]

# Separate variables for clarity
Y_lalonde = df_lalonde[outcome_lalonde]
T_lalonde = df_lalonde[treatment_lalonde]
X_lalonde = df_lalonde[covariates_lalonde]

print(f"\nTreatment distribution:")
print(f"Number of treated units: {T_lalonde.sum()}")
print(f"Number of control units: {len(df_lalonde) - T_lalonde.sum()}")
print(f"Treatment rate: {T_lalonde.mean():.1%}")

print(f"\nOutcome statistics:")
print(f"Mean earnings (treated): ${Y_lalonde[T_lalonde==1].mean():.2f}")
print(f"Mean earnings (control): ${Y_lalonde[T_lalonde==0].mean():.2f}")
print(f"Naive difference: ${Y_lalonde[T_lalonde==1].mean() - Y_lalonde[T_lalonde==0].mean():.2f}")

print(f"\nCovariates: {covariates_lalonde}")

# Display first few rows
print(f"\nFirst 5 rows of the dataset:")
df_lalonde.head()

## Applying Causal ML Methods to Real Data

Now let's apply all our causal ML methods to the Lalonde dataset to estimate both the ATE and individual treatment effects (CATEs). Real-world data often exhibits **genuine heterogeneity** - some individuals benefit more from treatment than others, making CATE analysis particularly valuable for policy targeting.

### S-Learner on Lalonde Data

In [ ]:
print("=== S-LEARNER ON LALONDE DATA ===")

# Create S-learner for Lalonde dataset
s_learner_lalonde = SLearner(
    overall_model=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    )
)

# Fit the model
s_learner_lalonde.fit(
    Y=Y_lalonde, 
    T=T_lalonde, 
    X=X_lalonde, 
    inference=BootstrapInference(n_bootstrap_samples=breps)
)

# Get ATE and confidence intervals
slearner_ate_lalonde = s_learner_lalonde.ate(X=X_lalonde)
slearner_ci_lower_lalonde, slearner_ci_upper_lalonde = s_learner_lalonde.ate_interval(X=X_lalonde, alpha=0.05)

# Get individual treatment effects (CATEs)
slearner_cate_lalonde = s_learner_lalonde.effect(X_lalonde)

print(f"S-learner ATE: ${slearner_ate_lalonde:.2f}")
print(f"95% Confidence Interval: [${slearner_ci_lower_lalonde:.2f}, ${slearner_ci_upper_lalonde:.2f}]")
print(f"CATE std: ${np.std(slearner_cate_lalonde):.2f}")
print(f"CATE range: [${np.min(slearner_cate_lalonde):.2f}, ${np.max(slearner_cate_lalonde):.2f}]")

In [ ]:
print("=== T-LEARNER ON LALONDE DATA ===")

# Create T-learner for Lalonde dataset
t_learner_lalonde = TLearner(
    models=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    )
)

# Fit the model
t_learner_lalonde.fit(
    Y=Y_lalonde, 
    T=T_lalonde, 
    X=X_lalonde, 
    inference=BootstrapInference(n_bootstrap_samples=breps)
)

# Get ATE and confidence intervals
t_learner_ate_lalonde = t_learner_lalonde.ate(X=X_lalonde)
t_learner_ci_lower_lalonde, t_learner_ci_upper_lalonde = t_learner_lalonde.ate_interval(X=X_lalonde, alpha=0.05)

# Get individual treatment effects (CATEs)
t_learner_cate_lalonde = t_learner_lalonde.effect(X_lalonde)

print(f"T-learner ATE: ${t_learner_ate_lalonde:.2f}")
print(f"95% Confidence Interval: [${t_learner_ci_lower_lalonde:.2f}, ${t_learner_ci_upper_lalonde:.2f}]")
print(f"CATE std: ${np.std(t_learner_cate_lalonde):.2f}")
print(f"CATE range: [${np.min(t_learner_cate_lalonde):.2f}, ${np.max(t_learner_cate_lalonde):.2f}]")

In [ ]:
print("=== X-LEARNER ON LALONDE DATA ===")

# Create X-learner for Lalonde dataset
x_learner_lalonde = XLearner(
    models=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    ),
    propensity_model=LGBMClassifier(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    ),
    cate_models=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    )
)

# Fit the model
x_learner_lalonde.fit(
    Y=Y_lalonde, 
    T=T_lalonde, 
    X=X_lalonde, 
    inference=BootstrapInference(n_bootstrap_samples=breps)
)

# Get ATE and confidence intervals
x_learner_ate_lalonde = x_learner_lalonde.ate(X=X_lalonde)
x_learner_ci_lower_lalonde, x_learner_ci_upper_lalonde = x_learner_lalonde.ate_interval(X=X_lalonde, alpha=0.05)

# Get individual treatment effects (CATEs)
x_learner_cate_lalonde = x_learner_lalonde.effect(X_lalonde)

print(f"X-learner ATE: ${x_learner_ate_lalonde:.2f}")
print(f"95% Confidence Interval: [${x_learner_ci_lower_lalonde:.2f}, ${x_learner_ci_upper_lalonde:.2f}]")
print(f"CATE std: ${np.std(x_learner_cate_lalonde):.2f}")
print(f"CATE range: [${np.min(x_learner_cate_lalonde):.2f}, ${np.max(x_learner_cate_lalonde):.2f}]")

In [ ]:
print("=== CAUSAL FOREST ON LALONDE DATA ===")

# Create Causal Forest for Lalonde dataset
causal_forest_lalonde = CausalForestDML(
    model_y=LGBMRegressor(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    ),
    model_t=LGBMClassifier(
        n_estimators=100,
        num_leaves=30,
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    ),
    n_estimators=500,  # n_estimators must be divisible by subforest_size
    subforest_size=5,  # Subforests to compute variance 
    max_depth=6,      # Reduced depth for faster execution
    min_samples_split=10,
    min_samples_leaf=5,
    cv=3,  # Cross-fitting folds (reduced for faster execution)
    discrete_treatment=True,
    n_jobs=-1,
    random_state=42
)

# Fit the model
causal_forest_lalonde.fit(
    Y=Y_lalonde, 
    T=T_lalonde, 
    X=X_lalonde
)

# Get ATE and confidence intervals
forest_ate_lalonde = causal_forest_lalonde.ate(X=X_lalonde)
forest_ci_lower_lalonde, forest_ci_upper_lalonde = causal_forest_lalonde.ate_interval(X=X_lalonde, alpha=0.05)

# Get individual treatment effects (CATEs)
cate_forest_lalonde = causal_forest_lalonde.effect(X_lalonde)

print(f"Causal Forest ATE: ${forest_ate_lalonde:.2f}")
print(f"95% Confidence Interval: [${forest_ci_lower_lalonde:.2f}, ${forest_ci_upper_lalonde:.2f}]")
print(f"CATE std: ${np.std(cate_forest_lalonde):.2f}")
print(f"CATE range: [${np.min(cate_forest_lalonde):.2f}, ${np.max(cate_forest_lalonde):.2f}]")

In [ ]:
print("=== DOUBLE MACHINE LEARNING ON LALONDE DATA ===")

# Create DML estimator for Lalonde dataset
cv_splitter_lalonde = KFold(n_splits=3, shuffle=True, random_state=42)  # Fewer folds for small dataset

est_dml_lalonde = LinearDML(
    model_y=LGBMRegressor(
        n_estimators=100, 
        num_leaves=30, 
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    ),
    model_t=LGBMClassifier(
        n_estimators=100, 
        num_leaves=30, 
        learning_rate=0.1,
        random_state=42,
        verbosity=-1
    ),
    cv=cv_splitter_lalonde,
    discrete_treatment=True,
    random_state=42
)

# Fit the model
est_dml_lalonde.fit(Y_lalonde, T=T_lalonde, X=X_lalonde)

# Get ATE and confidence intervals
ate_summary_lalonde = est_dml_lalonde.ate_inference(X=X_lalonde)
dml_ate_lalonde = ate_summary_lalonde.mean_point
dml_ci_lower_lalonde, dml_ci_upper_lalonde = ate_summary_lalonde.conf_int_mean(alpha=0.05)

# Get individual treatment effects (CATEs)
dml_cate_lalonde = est_dml_lalonde.effect(X_lalonde)

print(f"DML ATE: ${dml_ate_lalonde:.2f}")
print(f"P-value: {ate_summary_lalonde.pvalue():.4f}")
print(f"95% Confidence Interval: [${dml_ci_lower_lalonde:.2f}, ${dml_ci_upper_lalonde:.2f}]")
print(f"CATE std: ${np.std(dml_cate_lalonde):.2f}")
print(f"CATE range: [${np.min(dml_cate_lalonde):.2f}, ${np.max(dml_cate_lalonde):.2f}]")

## CATE Analysis: Heterogeneous Treatment Effects

Now let's examine the **heterogeneous treatment effects** in the real Lalonde data. This analysis reveals how different individuals respond to the job training program, providing insights for targeted policy implementation.

### Key Focus:
- **Individual-level effects**: How much does each person benefit from the job training program?
- **Policy targeting**: Which types of individuals should be prioritized for the program?
- **Method comparison**: How do different causal ML approaches capture treatment effect heterogeneity?

In [ ]:
# --- CATE Distribution Analysis for Lalonde Data ---

# Prepare data for CATE comparison (Lalonde dataset)
cate_data_lalonde = {
    'S-Learner': slearner_cate_lalonde.flatten(),
    'T-Learner': t_learner_cate_lalonde.flatten(),
    'X-Learner': x_learner_cate_lalonde.flatten(),
    'Causal Forest': cate_forest_lalonde.flatten(),
    'DML': dml_cate_lalonde.flatten()
}

# Corresponding ATE estimates for vertical lines (Lalonde dataset)
ate_estimates_dict_lalonde = {
    'S-Learner': slearner_ate_lalonde,
    'T-Learner': t_learner_ate_lalonde,
    'X-Learner': x_learner_ate_lalonde,
    'Causal Forest': forest_ate_lalonde,
    'DML': dml_ate_lalonde
}

# Create comprehensive comparison plot: Real Data (Lalonde)
fig, ax = plt.subplots(figsize=(15, 10))

# Color palette for consistency
colors = sns.color_palette("Set2", len(cate_data_lalonde))

# Plot KDE for each method on Lalonde data
for i, (method, cates) in enumerate(cate_data_lalonde.items()):
    # Create KDE plot
    sns.kdeplot(data=cates, label=f'{method} (Lalonde)', color=colors[i], 
                linewidth=2.5, alpha=0.8, ax=ax)
    
    # Add vertical line for model-specific ATE
    ax.axvline(x=ate_estimates_dict_lalonde[method], color=colors[i], linestyle='-', 
                alpha=0.7, linewidth=1.5)

# Calculate overall statistics for Lalonde data
all_cates_lalonde = np.concatenate([cates for cates in cate_data_lalonde.values()])
median_ate_lalonde = np.median([ate for ate in ate_estimates_dict_lalonde.values()])

# Add reference line for median ATE across all methods
ax.axvline(x=median_ate_lalonde, color='black', linestyle=':', linewidth=2, 
           label=f'Median ATE: ${median_ate_lalonde:.0f}', alpha=0.8)

# Set axis limits for better visualization
ax.set_ylim(0, 0.0005)     # Density scale appropriate for earnings

# Customize the plot
ax.set_xlabel('Conditional Average Treatment Effect (CATE) - Annual Earnings ($)', fontsize=14, fontweight='bold')
ax.set_ylabel('Density', fontsize=14, fontweight='bold')
ax.set_title('Distribution of Individual Treatment Effects (CATEs)\nLalonde NSW Dataset: Job Training Program', 
          fontsize=16, fontweight='bold', pad=20)

# Add legend
ax.legend(loc='upper right', fontsize=11, frameon=True, fancybox=True, shadow=True)

# Add grid for better readability
ax.grid(True, alpha=0.3)

# Set style
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.tight_layout()
plt.show()

# Print comprehensive CATE summary for Lalonde data
print("\n" + "="*100)
print("CATE DISTRIBUTION SUMMARY - LALONDE NSW DATASET")
print("="*100)
print(f"{'Method':<15} {'Mean CATE':<12} {'Median CATE':<12} {'Std CATE':<12} {'Min CATE':<12} {'Max CATE':<12} {'Range':<12}")
print("-"*100)

for method, cates in cate_data_lalonde.items():
    mean_cate = np.mean(cates)
    median_cate = np.median(cates)
    std_cate = np.std(cates)
    min_cate = np.min(cates)
    max_cate = np.max(cates)
    range_cate = max_cate - min_cate
    
    print(f"{method:<15} ${mean_cate:<11.0f} ${median_cate:<11.0f} ${std_cate:<11.0f} ${min_cate:<11.0f} ${max_cate:<11.0f} ${range_cate:<11.0f}")

print("="*100)

# Additional analysis for Lalonde data
print(f"\nLALONDE DATASET INSIGHTS:")
print(f"• Dataset size: {len(Y_lalonde)} individuals")
print(f"• Treatment rate: {T_lalonde.mean():.1%}")
print(f"• Average earnings (control): ${Y_lalonde[T_lalonde==0].mean():.0f}")
print(f"• Average earnings (treated): ${Y_lalonde[T_lalonde==1].mean():.0f}")
print(f"• Naive difference: ${Y_lalonde[T_lalonde==1].mean() - Y_lalonde[T_lalonde==0].mean():.0f}")

print(f"\nHETEROGENEITY ANALYSIS:")
print("• Positive CATEs suggest the individual would benefit from the job training program")
print("• Negative CATEs suggest the program might actually hurt that individual's earnings")
print("• Large standard deviations indicate substantial heterogeneity across individuals")
print("• Range shows the difference between most and least benefited individuals")